<a href="https://colab.research.google.com/github/zhangwenhe1007/Qua-NYUAD2025/blob/main/greedy_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Reload dataset
filtered_df = pd.read_csv("filtered_DLRM_trace.csv")

# Take a manageable subset for demonstration, e.g., first 100 jobs
subset_df = filtered_df.head(2000).copy()

# Group by app_name and preserve order within each app group
subset_df_sorted = subset_df.sort_values(by=['app_name' , 'creation_time'])

# Simulated refined machine specs
NUM_MACHINES = 30
cap_cpu = 256
cap_gpu = 8
cap_memory = 1024

# Initialize machines with new specs
machines_refined = [{
    'id': f'machine_{j}',
    'remaining_cpu': cap_cpu,
    'remaining_gpu': cap_gpu,
    'remaining_memory': cap_memory,
    'assigned_jobs': []
} for j in range(NUM_MACHINES)]

# Greedy FCFS-style assignment
for idx, row in subset_df_sorted.iterrows():
    for machine in machines_refined:
        if (row['cpu_request'] <= machine['remaining_cpu'] and
            row['gpu_request'] <= machine['remaining_gpu'] and
            row['memory_request'] <= machine['remaining_memory']):

            # Assign job
            machine['remaining_cpu'] -= row['cpu_request']
            machine['remaining_gpu'] -= row['gpu_request']
            machine['remaining_memory'] -= row['memory_request']
            machine['assigned_jobs'].append({
                'instance_sn': row['instance_sn'],
                'app_name': row['app_name'],
                'estimated_runtime': row['estimated_runtime'],
                'cpu_request': row['cpu_request'],
                'gpu_request': row['gpu_request'],
                'memory_request': row['memory_request']
            })
            break  # Move to next job once assigned

# Prepare detailed summary
machines_assignment_fcfs = [{
    'machine_id': m['id'],
    'num_jobs': len(m['assigned_jobs']),
    'assigned_job_ids': [job['instance_sn'] for job in m['assigned_jobs']],
    'remaining_cpu': m['remaining_cpu'],
    'remaining_gpu': m['remaining_gpu'],
    'remaining_memory': m['remaining_memory'],
    'total_runtime': sum(job['estimated_runtime'] for job in m['assigned_jobs'])
} for m in machines_refined]

# Total tasks scheduled
total_tasks_fcfs = sum(len(m['assigned_jobs']) for m in machines_refined)

machines_assignment_fcfs, total_tasks_fcfs

